In [98]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
train_file = './resources/MC2-train.arff'
pred_file = './resources/MC2-predict.arff'

In [39]:
def parse_to_df(file):
    with open(file) as f:
        mode = ''
        col_names = []
        col_types = []
        data = []
        for line in f.readlines():
            line_splits = line.rstrip().split(' ')

            if '@relation' in line:
                mode = '@relation'
            elif '@data' in line:
                mode = '@data'

            if mode == '@relation':
                line_splits = line.rstrip().split(' ')
                if len(line_splits) == 3:
                    col_name = line_splits[1]
                    col_type = line_splits[2]

                    col_names.append(col_name)
                    col_types.append(col_type)


            if mode == '@data':
                line_splits = line.rstrip().split(',')
                cast_vals = []
                if len(line_splits) > 1:
                    for idx, val in enumerate(line_splits):
                        cast_vals.append(cast(val, col_types[idx]))

                    data.append(tuple(cast_vals))

    return pd.DataFrame(data, columns=col_names)

In [40]:
def cast(val, col_type):
    if (val == '?'):
        return np.nan
    elif (col_type == '{Y,N}'):
        return True if val == 'Y' else False
    elif (col_type == 'numeric'):
        return float(val)
    else:
        return val

In [41]:
train_df = parse_to_df(train_file)
train_df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PATHOLOGICAL_COMPLEXITY,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,2.0,3.0,0.0,2.0,7.0,4.0,2.0,0.18,2.0,2.0,...,0.09,11.0,15.0,6.0,8.0,22.0,1.0,50.00,11.0,False
1,1.0,3.0,1.0,1.0,0.0,4.0,2.0,0.13,2.0,2.0,...,0.11,12.0,19.0,8.0,10.0,18.0,1.0,6.67,15.0,False
2,6.0,5.0,1.0,4.0,5.0,8.0,3.0,0.20,4.0,2.0,...,0.10,25.0,38.0,12.0,11.0,29.0,1.0,45.00,15.0,False
3,6.0,5.0,1.0,4.0,3.0,8.0,3.0,0.20,4.0,2.0,...,0.12,26.0,38.0,10.0,11.0,26.0,1.0,38.89,15.0,False
4,2.0,3.0,0.0,0.0,1.0,4.0,2.0,0.29,2.0,2.0,...,0.17,7.0,13.0,5.0,9.0,12.0,1.0,12.50,7.0,False


In [43]:
pred_df = parse_to_df(pred_file)
pred_df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PATHOLOGICAL_COMPLEXITY,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,5.0,3.0,0.0,0.0,3.0,4.0,2.0,0.29,2.0,2.00,...,0.12,15.0,14.0,10.0,6.0,17.0,1.0,30.00,7.0,False
1,5.0,3.0,0.0,0.0,3.0,4.0,2.0,0.29,2.0,2.00,...,0.12,15.0,15.0,10.0,6.0,17.0,1.0,30.00,7.0,False
2,5.0,3.0,0.0,0.0,3.0,4.0,2.0,0.29,2.0,2.00,...,0.12,14.0,14.0,10.0,6.0,17.0,1.0,30.00,7.0,False
3,30.0,69.0,17.0,4.0,52.0,118.0,35.0,0.27,54.0,2.19,...,0.16,145.0,218.0,63.0,24.0,213.0,1.0,30.94,129.0,False
4,2.0,1.0,0.0,0.0,3.0,0.0,1.0,0.25,0.0,NaN,...,0.09,4.0,7.0,3.0,6.0,11.0,1.0,42.86,4.0,False


In [86]:
X_t = train_df.iloc[:, 0:-1].fillna(-1)
y_t = train_df.iloc[:, -1]

In [89]:
mask = ~y_t.isna()

In [91]:
X_t = X_t[mask]
y_t = y_t[mask]

In [99]:
reg = LogisticRegression().fit(X_t, y_t)

/Users/hao/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [100]:
reg.score(X_t, y_t)

0.7872340425531915